# CrewAI + Code Scalpel: Refactoring Demo

This notebook demonstrates how to use CrewAI with Code Scalpel for automated code refactoring tasks.

## Overview

We'll create a CrewAI crew that uses Code Scalpel's AST analysis capabilities to:
1. Analyze code for issues and improvements
2. Identify security vulnerabilities
3. Suggest and apply refactoring improvements

## Setup

First, let's import the necessary modules and set up our environment.

In [ ]:
import asyncio
import sys

# Add src to path if running from examples directory
sys.path.insert(0, '../src')

# Import Code Scalpel integrations
from integrations import CrewAIScalpel

print("Code Scalpel imported successfully!")

## Initialize CrewAIScalpel

Create an instance of CrewAIScalpel which provides AST analysis tools for the crew.

In [ ]:
# Initialize the CrewAI Scalpel wrapper
scalpel = CrewAIScalpel(cache_enabled=True)

# Get available tools for CrewAI
tools = scalpel.get_crewai_tools()
print("Available Scalpel tools for CrewAI:")
for tool in tools:
    print(f"  - {tool['name']}: {tool['description'][:60]}...")

## Sample Code to Refactor

Let's define some Python code with common issues that we want to analyze and refactor.

In [ ]:
# Sample code with various issues
sample_code = '''
def ProcessUserData(userData, configSettings):
    """Process user data based on config settings."""
    if userData is not None:
        if configSettings is not None:
            if configSettings.get("validate"):
                if userData.get("email"):
                    if "@" in userData.get("email"):
                        result = eval(f"process_{configSettings['mode']}(userData)")  # Security issue!
                        return result
    return None


def calculateDiscount(price, discountPercent, taxRate, handlingFee, membershipLevel):
    """Calculate final price with discount."""
    if price > 0:
        if discountPercent > 0:
            if discountPercent <= 100:
                discounted = price * (1 - discountPercent / 100)
                if taxRate > 0:
                    withTax = discounted * (1 + taxRate / 100)
                    final = withTax + handlingFee
                    return final
    return price
'''

print("Sample code to analyze:")
print(sample_code)

## Step 1: Analyze the Code

Use the CrewAIScalpel to perform comprehensive code analysis.

In [ ]:
# Perform code analysis
async def analyze_code():
    result = await scalpel.analyze_async(sample_code)
    return result

# Run analysis
analysis_result = asyncio.get_event_loop().run_until_complete(analyze_code())

print("\n=== Analysis Results ===")
print(f"Success: {analysis_result.success}")
print("\nAnalysis Summary:")
print(f"  - Total Issues Found: {analysis_result.analysis.get('total_issues', 0)}")
print(f"  - Style Issues: {analysis_result.analysis.get('style_issues', 0)}")
print(f"  - Security Issues: {analysis_result.analysis.get('security_issues', 0)}")

In [ ]:
# Display detailed issues
print("\n=== Detailed Issues ===")
for i, issue in enumerate(analysis_result.issues, 1):
    print(f"\n{i}. Type: {issue.get('type', 'unknown')}")
    if issue.get('category'):
        print(f"   Category: {issue['category']}")
    if issue.get('details'):
        print("   Details:")
        for detail in issue.get('details', []):
            print(f"     - {detail}")
    if issue.get('function'):
        print(f"   Dangerous function: {issue['function']}")
    if issue.get('line'):
        print(f"   Line: {issue['line']}")

In [ ]:
# Display suggestions
print("\n=== Improvement Suggestions ===")
for i, suggestion in enumerate(analysis_result.suggestions, 1):
    print(f"{i}. {suggestion}")

## Step 2: Security Scan

Perform a focused security analysis to identify vulnerabilities.

In [ ]:
# Perform security-focused analysis
async def security_scan():
    result = await scalpel.analyze_security_async(sample_code)
    return result

security_result = asyncio.get_event_loop().run_until_complete(security_scan())

print("\n=== Security Analysis ===")
print(f"Risk Level: {security_result['risk_level'].upper()}")
print(f"\nVulnerabilities Found: {len(security_result['issues'])}")

for issue in security_result['issues']:
    print(f"\n  - Type: {issue.get('type')}")
    if issue.get('function'):
        print(f"    Function: {issue['function']}")
    if issue.get('line'):
        print(f"    Line: {issue['line']}")

print("\nRecommendations:")
for rec in security_result['recommendations']:
    print(f"  • {rec}")

## Step 3: Refactor the Code

Use CrewAIScalpel to generate refactored code based on the analysis.

In [ ]:
# Perform refactoring
async def refactor_code():
    result = await scalpel.refactor_async(
        sample_code, 
        task_description="Improve code quality and fix security issues"
    )
    return result

refactor_result = asyncio.get_event_loop().run_until_complete(refactor_code())

print("\n=== Refactoring Results ===")
print(f"Success: {refactor_result.success}")

if refactor_result.refactored_code:
    print("\n--- Refactored Code ---")
    print(refactor_result.refactored_code)

## Step 4: Creating a CrewAI Agent with Scalpel Tools

Now let's demonstrate how to integrate Scalpel with a CrewAI agent.

In [ ]:
# Example of how to use Scalpel tools with CrewAI
# Note: This requires CrewAI and an LLM configuration

def create_refactoring_crew():
    """
    Example function showing how to create a CrewAI crew with Scalpel tools.
    
    Note: In a real scenario, you would need:
    - CrewAI installed: pip install crewai
    - An LLM configured (e.g., OpenAI API key)
    """
    try:
        from crewai import Agent, Crew, Task
        
        # Initialize Scalpel
        scalpel = CrewAIScalpel()
        scalpel_tools = scalpel.get_crewai_tools()
        
        # Create a code review agent
        code_reviewer = Agent(
            role="Senior Code Reviewer",
            goal="Analyze code for quality issues and security vulnerabilities",
            backstory="Expert Python developer with years of experience in code review",
            tools=[tool['func'] for tool in scalpel_tools],
            verbose=True
        )
        
        # Create a refactoring task
        refactoring_task = Task(
            description="""
            Analyze the provided Python code and:
            1. Identify all code quality issues
            2. Find security vulnerabilities
            3. Suggest refactoring improvements
            4. Provide a summary report
            """,
            agent=code_reviewer
        )
        
        # Create the crew
        analysis_crew = Crew(
            agents=[code_reviewer],
            tasks=[refactoring_task],
            verbose=True
        )
        
        return analysis_crew
        
    except ImportError as e:
        print(f"CrewAI not fully configured: {e}")
        print("\nTo use CrewAI, ensure you have:")
        print("  1. pip install crewai")
        print("  2. Set OPENAI_API_KEY environment variable")
        return None

# Try to create the crew (for demonstration)
print("Attempting to create CrewAI refactoring crew...")
crew = create_refactoring_crew()

## Step 5: Using the MCP Server

Code Scalpel also provides an MCP server for HTTP-based analysis.

In [ ]:
# Demonstrate MCP server usage (using test client)
from integrations.mcp_server import create_app

# Create test client
app = create_app()
client = app.test_client()

# Make an analysis request
response = client.post('/analyze', json={'code': sample_code})
result = response.get_json()

print("\n=== MCP Server Response ===")
print(f"Status: {response.status_code}")
print(f"Success: {result.get('success')}")
print(f"Processing Time: {result.get('processing_time_ms'):.2f}ms")
print(f"\nIssues: {len(result.get('issues', []))}")
print(f"Suggestions: {len(result.get('suggestions', []))}")

## Summary

This notebook demonstrated:

1. **CrewAIScalpel Integration** - Using Code Scalpel's AST analysis with CrewAI agents
2. **Code Analysis** - Identifying style issues, security vulnerabilities, and improvement opportunities
3. **Security Scanning** - Focused security analysis with risk assessment
4. **Refactoring** - Automated code improvement suggestions
5. **MCP Server** - HTTP API for agent queries

### Key Features:

- **Async Support**: All analysis methods support async/await for integration with async frameworks
- **Performance**: Analysis completes in milliseconds, well under the 2s requirement
- **Comprehensive Analysis**: Covers style, naming conventions, security, and code structure
- **Tool Integration**: Ready-to-use tools for CrewAI and other agent frameworks

In [ ]:
print("\n✓ Demo completed successfully!")